In [1]:
import numpy as np
import pandas as pd
import datetime

In [5]:
!pip install xlrd

    100% |████████████████████████████████| 112kB 3.8MB/s 
You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
def diff_datetime_time(time_1, time_2):
    # https://stackoverflow.com/questions/43305577/python-calculate-the-difference-between-two-datetime-time-objects
    datetime_1 = datetime.datetime.combine(datetime.date.today(), time_1)
    datetime_2 = datetime.datetime.combine(datetime.date.today(), time_2)
    if time_1 < time_2:
        return datetime_2 - datetime_1
    else:
        return datetime_1 - datetime_2

In [35]:
class DiscrepancyIdentifier:
    
    def __init__(self, *, file_1, file_2, formatter_1, formatter_2,
                 threshold_begin, threshold_end, col_begin='Time Begin',
                 col_end='Time End'):
        """
        `file_1` and `file_2` are the names of the .xlxs files
        `formatter_1` is a function that takes in the output of 
        `pd.read_excel(file_1, dtype={'Time Begin': str, 'Time End: str'})`
        (a pandas dataframe) and formats the time columns to work with
        the rest of the class, i.e. both should datetime times;
        `formatter_2` should be the same for `file_2`
        """
        self.col_begin = col_begin
        self.col_end = col_end
        
        self.df_1 = pd.read_excel(
            file_1,
            dtype={self.col_begin: str, self.col_end: str}
        )
        self.df_2 = pd.read_excel(
            file_2,
            dtype={self.col_begin: str, self.col_end: str}
        )
        
        formatter_1(self.df_1)
        formatter_2(self.df_2)
        
        self.threshold_begin = datetime.timedelta(seconds=threshold_begin)
        self.threshold_end = datetime.timedelta(seconds=threshold_end)

        # list of tuples with matching indices
        # for now, the matches need not be 1-1 but this behavior
        # should be modified appropriately, especially since there
        # are many second to second exchanges which occur
        self.match_begin = list()
        self.match_end = list()
        self.match_both = None
        
        self.matches_found = False
        self.discrepancy_df = None
        
    def find_matches(self):
        if self.matches_found:
            print('matches already found')
            return
        for col, match_list, threshold in zip([self.col_begin, self.col_end], [self.match_begin, self.match_end], [self.threshold_begin, self.threshold_end]):
            for i, time_1 in enumerate(self.df_1[col]):
                subsequent_required = False
                for j, time_2 in enumerate(self.df_2[col]):
                    if diff_datetime_time(time_1, time_2) <= threshold:
                        match_list.append((i, j))
                        subsequent_required = True
                    elif subsequent_required:
                        subsequent_required = False
                        break
                    # exits if subsequent time is not also a match
                    else:
                        continue

        self.match_both = sorted(list(set(self.match_begin) & set(self.match_end)))
        
        self.matches_found = True
        print('matches found')
        
    def identify_discrepancies(self):
        if not self.matches_found:
            self.find_matches()
        if not self.discrepancy_df:
            match_both_arr = np.array(self.match_both)
            # NaN are replaced with False here so NaN == NaN when normally doesn't
            df_1_match_both = self.df_1.iloc[match_both_arr[:, 0]].reset_index(drop=True).replace(np.NaN, False)
            df_2_match_both = self.df_2.iloc[match_both_arr[:, 1]].reset_index(drop=True).replace(np.NaN, False)
            self.discrepancy_df = df_1_match_both == df_2_match_both
            print('discrepancies found; see discrepancy dataframe with .result')
        else:
            print('discrepancies already identified; see discrepancy dataframe with .result')
        
    @property
    def result(self):
        return self.discrepancy_df
        

In [36]:
def formatter_a(df):
    for col in ['Time Begin', 'Time End']:
        df[col] = pd.to_datetime(df[col]).map(lambda dt: dt.time())

In [37]:
def formatter_b(df):
    for col in ['Time Begin', 'Time End']:
        df[col] = df[col].map(lambda time: datetime.datetime.strptime(time, '%M:%S').time())

In [38]:
di = DiscrepancyIdentifier(
    file_1='HeatherGray_Physics.xlsx',
    file_2='HeatherGray2_Physics.xlsx',
    formatter_1=formatter_a,
    formatter_2=formatter_b,
    threshold_begin=3,
    threshold_end=3
)

In [39]:
di.identify_discrepancies()

matches found
discrepancies found; see discrepancy dataframe with .result


In [40]:
di.result

,Time Begin,Time End,Who's Speaking?,Acknowledged?,Attempted Interruption?,Follow up?,Scientific Comment?,Non-scientific comment?,Positive comment?,Clarifying question?,Furthering question?,Critical element?,Ad hominem?,Self referential?
0,True,False,True,True,True,True,True,True,True,True,True,True,True,True
1,False,True,True,False,False,True,True,True,True,True,True,True,True,True
2,True,False,True,True,False,True,True,True,True,True,True,True,True,True
3,True,False,True,True,False,True,True,True,True,True,True,True,True,False
4,True,False,True,True,False,True,True,True,True,True,True,True,True,True
5,False,False,True,False,False,True,False,True,True,False,True,True,True,True
6,True,False,True,True,False,True,True,True,True,True,True,True,True,True
7,True,False,True,True,False,True,False,True,True,True,True,True,True,False
8,True,False,True,True,False,True,True,True,True,True,True,True,True,True
9,False,False,False,False,True,False,False,False,False,False,False,False,False,False
